In [ ]:
#@title init
%%writefile __init__.py
import copy
serverconnections = [{'servername':'msIA',
  'headers' : {'Authorization':'Token abe8934572784dea'},
  'root_url' : 'https://msia.escriptorium.fr'},
 {'servername':'YOUR_SERVERNAME2',
  'headers' : {'Authorization':'Token YOURTOKEN2'},
  'root_url' : 'https://...'},
  {'servername':'YOUR_SERVERNAME3',
  'headers' : {'Authorization':'Token YOURTOKEN3'},
  'root_url' : 'https://...'}]

for serverconnection in serverconnections:
    serverconnection['headersbrief'] = copy.deepcopy(serverconnection['headers'])
    serverconnection['headers']['Content-type'] = 'application/json'
    serverconnection['headers']['Accept']= 'application/json'

if __name__ == '__main__':
  print("<> Running the module as a script! <>")

In [ ]:
#@title basic
#%%writefile basic_api_functions.py

# from basic_api_functions import create_new_document, create_transcription_levels, upload_image, loop_through_itempages, get_part_pk_list, get_all_lines_of_region,get_line_pk_list, get_linetranscription_pk_list, get_inhabited_region_pk_list, get_region_pk_list, get_region_box_list, get_page_transcription, get_document_segmentation_ontology, get_transcription_levels, get_set_of_all_characters_in_transcription_level, get_list_of_image_names, move_part, repolygonize, reorder,check_point_in_image,get_centroid,get_angle,rotateNP,line_intersection

import json
import requests
import os
import io
from requests.compat import urljoin
from shapely.geometry import Point, Polygon
from numpy import rad2deg,deg2rad,array,atleast_2d,cos,sin,arctan2,squeeze,cross,linalg
from statistics import mean
from PIL import Image
from math import sqrt

def search(pk, list_of_dictionaries):
    result_list=[element for element in list_of_dictionaries if element['pk'] == pk]
    if len(result_list)==1:
      return result_list  [0]
    elif len(result_list)==0:
      return None
    else:
      return result_list

def search_any(key, value, list_of_dictionaries):
    return [element for element in list_of_dictionaries if element[key] == value]


def get_idx(pk, list_of_dictionaries):
  result_list=[n for n,element in enumerate(list_of_dictionaries) if element['pk'] == pk]
  if len(result_list)==1:
    return result_list[0]
  elif len(result_list)==0:
    return None
  else:
    return result_list

def check_token_contains(key,chars,list_of_dictionaries):
  return [n for n,element in enumerate(list_of_dictionaries) if any((c in chars) for c in element[key])]

def get_serverinfo(servername,serverconnections):
  serverconnection=search_any('servername', servername, serverconnections)[0]
  print('switching to ',servername)
  headers = serverconnection['headers']
  headersbrief = serverconnection['headersbrief']
  root_url = serverconnection['root_url']
  return root_url,headers,headersbrief

def dist2points(x1,y1,x2,y2):
  dist=sqrt((x2 - x1)**2 + (y2 - y1)**2)
  return dist

def loop_through_itempages_and_get_all(base_url,suffix=''):
    result_list=[]
    def get_page(page,result_list):
        data = False
        #url = urljoin(base_url, '?page=%d' % page)
        if suffix=='':
          url = base_url+'?page='+str(page)
        else:
          url = base_url+'?page='+str(page)+'&'+suffix
        res = requests.get(url, headers=headers)
        #print(res.json())
        try:
            data = res.json()
        except json.decoder.JSONDecodeError as e:
            print('exception')
            print(res)
        else:
          if data:
         #   print(data)
            result_list += [item for item in data['results']]
        if data:
          if data['next']:
            get_page(page+1,result_list)
    get_page(1,result_list)
    return(result_list)

def get_all_item_info(base_url):
    result_list=[]
    def get_page(page,result_list):
        url = base_url+'?page='+str(page)
        res = requests.get(url, headers=headers)
        try:
            data = res.json()
        except json.decoder.JSONDecodeError as e:
            data = []
            print(res)
        else:
            #print('empty')
            #data = []
            if data:
              print(data)
              result_list += [item for item in data['results']]
        if data:
          if (data['next']):
            get_page(page+1,result_list)
    get_page(1,result_list)
    return(result_list)

def find_string_in_page_transcription(doc_pk,part_pk,tr_level,query_string):
    tr_url = root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+'/transcriptions/?transcription='+str(tr_level)+'&'
    r=requests.get(tr_url,headers=headers).json()
    for n,line in enumerate(r['results']):
      if line['content'].find(query_string)>-1:
        print(part_pk,'line :',n)

def get_basic_info(doc_pk):
  '''
  input: document ID
  output:
  part_list (list of all document parts),
  transcription_level_list (list of different transcription levels)
  region_type_list, line_type_list (segmentation ontologies)
  '''
  part_list=get_part_pk_list(doc_pk)
  print('Document:', doc_pk,' with ',len(part_list),' parts')
  region_type_list,line_type_list=get_document_segmentation_ontology(doc_pk)
  transcription_level_list=get_transcription_levels(doc_pk)
  return part_list,transcription_level_list,region_type_list,line_type_list

def get_margin_and_paratext_type_pks(doc_pk):
    region_type_list,line_type_list=get_document_segmentation_ontology(doc_pk)
    for region_type in region_type_list:
        if region_type['name']=='Margin':
            margin_type_pk=region_type['pk']
        elif region_type['name']=='Paratext':
            paratext_type_pk=region_type['pk']
    return margin_type_pk,paratext_type_pk

def create_new_document(doc_name,script_pk="Hebrew",read_direction="rtl"):
    url=root_url+'api/documents/'
    data=json.dumps({"name": doc_name,"main_script": script_pk, "read_direction":read_direction,"valid_block_types": [{"pk": 3,"name": "Commentary"},{"pk": 4,"name": "Illustration"
},{"pk": 2,"name": "Main"},{"pk": 1,"name": "Title"}]})
    res = requests.post(url,headers=headers,data=data)
    print(res.status_code, res.content)
    jsonResponse=json.loads(res.content.decode('utf-8'))
    doc_pk=jsonResponse['pk']
    return doc_pk

def create_transcription_levels(doc_pk,tr_name):
    url=root_url+'api/documents/'+str(doc_pk)+'/transcriptions/'
    transcription_level_list = requests.get(url, headers=headers).json()
    print('existing transcription levels: ',transcription_level_list)
    datajson={'name':tr_name}
    data=json.dumps(datajson)
    res=requests.post(url,headers=headers,data=data)
    jsonResponse=json.loads(res.content.decode('utf-8'))
    tr_pk=jsonResponse['pk']
    return tr_pk

def upload_image(doc_pk,dirname,fname):

    url = root_url+'api/documents/'+str(doc_pk)+'/parts/'
#mydir=(r"A:\openITI\testset")
#myfile='15.png'
    file = os.path.join(dirname, fname)
    with open(file, 'rb') as fh:
        data={'name':fname}
        res = requests.post(url,data=data, files={'image': fh},headers=headers)
    print(res.status_code, res.content)
    jsonResponse=json.loads(res.content.decode('utf-8'))
    part_pk=jsonResponse['pk']
    return part_pk

def page_height_width(doc_pk,part_pk):
  parts_url = urljoin(root_url,'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+'/')
  res = requests.get(parts_url,headers=headers).json()
  pagewidth=res['image']['size'][0]
  pageheight=res['image']['size'][1]
  return pageheight,pagewidth

def get_img_from_url(img_url):
  res=requests.get(img_url)
  img = Image.open(io.BytesIO(res.content))
  return(img)

def rotate_img(doc_pk,part_pk,angle):
  url=root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+'/rotate/'
  data={'angle':angle}
  print(data)
  res=requests.post(url=url,headers=headers,data=json.dumps(data))
  print(res)

def get_doc_list_from_instance():
  doc_url=urljoin(root_url, '/api/documents/')
  doc_info=requests.get(doc_url,headers=headers).json()
  doc_pk_list=loop_through_itempages(doc_url,'pk','results')
  doc_name_list=loop_through_itempages(doc_url,'name','results')
  doc_img_nu_list=loop_through_itempages(doc_url,'parts_count','results')
  f = open("results.tsv",mode='wt')
  for n,doc_pk in enumerate(doc_pk_list):
    doc_name=doc_name_list[n]
    doc_img_nu=doc_img_nu_list[n]
    f.write(str(doc_pk)+'\t'+doc_name+'\t'+str(doc_img_nu)+'\n')
  f.close()

def get_part_pk_list(doc_pk):
    parts_url=urljoin(root_url, '/api/documents/%d/parts/' % int(doc_pk))
    return(loop_through_itempages(parts_url,'pk','results'))

def get_imgname_list(doc_pk):
    parts_url=urljoin(root_url, '/api/documents/%d/parts/' % int(doc_pk))
    return(loop_through_itempages(parts_url,'filename','results'))

def get_line_pk_list(doc_pk,part_pk):
    lines_url = root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+'/lines/'
    return(loop_through_itempages(lines_url,'pk','results'))

def get_all_baselines_of_part(doc_pk,part_pk):
    lines_url = root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+'/lines/'
    return(loop_through_itempages(lines_url,'baseline','results'))

def get_linetranscription_pk_list(doc_pk, part_pk,tr_pk):
    tr_url = root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+'/transcriptions/'
    return(loop_through_itempages(tr_url,'pk','results'))

def get_inhabited_region_pk_list(doc_pk,part_pk):
    lines_url = root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+'/lines/'
    return(list(set(loop_through_itempages(lines_url,'region','results'))))

def get_region_pk_list(doc_pk,part_pk):
    regions_url = root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+'/blocks/'
    return(loop_through_itempages(regions_url,'pk','results'))


def get_region_pk_list_of_type(doc_pk,part_pk,region_type):
    regions_url = root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+'/blocks/'
    region_list=[]
    def get_page(page,region_list):
        url = urljoin(regions_url, '?page=%d' % page)
        res = requests.get(url, headers=headers)
        try:
            data = res.json()
        except json.decoder.JSONDecodeError as e:
            print(res)
        else:
            region_list += [region['pk'] for region in data['results'] if region['typology']==region_type]
        if data['next']:
            get_page(page+1,region_list)
    get_page(1,region_list)
    return(region_list)

def get_regions_and_lines(doc_pk,part_pk,tr_level):
  lines_url=root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+'/lines/'
  lines=loop_through_itempages_and_get_all(lines_url)
  regions_url=root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+'/blocks/'
  regions=loop_through_itempages_and_get_all(regions_url)
  tr_url = root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+'/transcriptions/'
  tr_suffix = 'transcription='+str(tr_level)
  #transcriptions=requests.get(tr_url,headers=headers).json()
  transcriptions=loop_through_itempages_and_get_all(tr_url,tr_suffix)
  return regions,lines,transcriptions

def get_main_region_pk_list(doc_pk,part_pk):
    regions_url = root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+'/blocks/'
    region_list=[]
    def get_page(page,region_list):
        url = urljoin(regions_url, '?page=%d' % page)
        res = requests.get(url, headers=headers)
        try:
            data = res.json()
        except json.decoder.JSONDecodeError as e:
            print(res)
        else:
            region_list += [region['pk'] for region in data['results'] if region['typology']==2]
        if data['next']:
            get_page(page+1,region_list)
    get_page(1,region_list)
    return(region_list)

def loop_through_itempages(base_url,query1,query2):
    result_list=[]
    def get_page(page,result_list):
        data=None
        url = base_url+ '?page='+str(page)
        res = requests.get(url, headers=headers)
        #print(res.json())
        try:
            data = res.json()
        except json.decoder.JSONDecodeError as e:
            print(res)
        else:
          if data and not(data=={'detail': 'You do not have permission to perform this action.'}):
         #   print(data)
            result_list += [item[query1] for item in data[query2]]
        if data and not(data=={'detail': 'You do not have permission to perform this action.'}):
          if data['next']:
            get_page(page+1,result_list)
    get_page(1,result_list)
    return(result_list)

def get_region_box_list(doc_pk,part_pk):
    regions_url = root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+'/blocks/'
    return(loop_through_itempages(regions_url,'box','results'))

def get_page_transcription(doc_pk,part_pk,tr_level):
    tr_url = root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+'/transcriptions/?transcription='+str(tr_level)+'&'
    return(loop_through_itempages(tr_url,'content','results'))

def get_page_transcription_line_pks(doc_pk,part_pk,tr_level):
    tr_url = root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+'/transcriptions/?transcription='+str(tr_level)+'&'
    return(loop_through_itempages(tr_url,'line','results'))

def get_all_lines_of_region(doc_pk,part_pk,region_pk):
    lines_url = root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+'/lines/'
    line_list=[]
    def get_page(page,line_list):
        url = urljoin(lines_url, '?page=%d' % page)
        res = requests.get(url, headers=headers)
        try:
            data = res.json()
        except json.decoder.JSONDecodeError as e:
            print(res)
        else:
            line_list += [[line['pk'],line['order'],line['baseline'],line['typology']] for line in data['results'] if line['region']==region_pk]
        if data['next']:
            get_page(page+1,line_list)
    get_page(1,line_list)
    return(line_list)

def get_all_line_pks_of_region(doc_pk,part_pk,region_pk):
    lines_url = root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+'/lines/'
    line_list=[]
    def get_page(page,line_list):
        url = urljoin(lines_url, '?page=%d' % page)
        res = requests.get(url, headers=headers)
        try:
            data = res.json()
        except json.decoder.JSONDecodeError as e:
            print(res)
        else:
            line_list += [line['pk'] for line in data['results'] if line['region']==region_pk]
        if data['next']:
            get_page(page+1,line_list)
    get_page(1,line_list)
    return(line_list)


def get_all_lines_of_part(doc_pk,part_pk):
    lines_url = root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+'/lines/'
    line_list=[]
    def get_page(page,line_list):
        url = urljoin(lines_url, '?page=%d' % page)
        res = requests.get(url, headers=headers)
        try:
            data = res.json()
        except json.decoder.JSONDecodeError as e:
            print(res)
        else:
            line_list += [[line['pk'],line['order'],line['baseline'],line['typology'],line['region']] for line in data['results']]
        if data['next']:
            get_page(page+1,line_list)
    get_page(1,line_list)
    return(line_list)

def get_all_full_lines_of_part(doc_pk,part_pk):
    lines_url = root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+'/lines/'
    line_info=[]
    def get_page(page,line_info):
        url = urljoin(lines_url, '?page=%d' % page)
        res = requests.get(url, headers=headers)
        try:
            data = res.json()
        except json.decoder.JSONDecodeError as e:
            print(res)
        else:
            line_info += data['results']
        if data['next']:
            get_page(page+1,line_info)
    get_page(1,line_info)
    return(line_info)

def get_document_segmentation_ontology(doc_pk):
    print('get document segmentation ontology for document: ',doc_pk)
    document_base_json=requests.get(root_url+'api/documents/'+str(doc_pk)+'/',headers=headers).json()
    print(document_base_json)
    region_type_list=document_base_json['valid_block_types']
    line_type_list=document_base_json['valid_line_types']
    margin_type_pk=-1
    paratext_type_pk=-1
    for region_type in region_type_list:
        print(region_type)
    for line_type in line_type_list:
        print(line_type)
    return region_type_list,line_type_list

def get_transcription_levels(doc_pk):
    print('get transcription levels for document: ',doc_pk)
    url=root_url+'api/documents/'+str(doc_pk)+'/transcriptions/'
    transcription_level_list = requests.get(url, headers=headers).json()
    print(transcription_level_list)
    return transcription_level_list

def get_page_transcription_tr_pks(doc_pk,part_pk,tr_level):
    tr_url = root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+'/transcriptions/?transcription='+str(tr_level)+'&'
    return(loop_through_itempages(tr_url,'pk','results'))

def get_set_of_all_characters_in_transcription_level(doc_pk,part_list,tr_level):
    print('get char set of document for document: ',doc_pk)
    chars = set()
    for n,part_pk in enumerate(part_list):
        if n % 100 == 0:
            print(n,' parts finished')
        transcriptions_this_part=get_page_transcription(doc_pk,part_pk,tr_level)
        for line in transcriptions_this_part:
            chars=chars.union(set(line))
    chars=list(chars)
    return chars

def get_list_of_image_names(doc_pk,part_list):
    print('order part_pk imgname')
    for n,part_pk in enumerate(part_list):
        url=root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)
        res = requests.get(url,headers=headers).json()
    #print('done')
        print(n+1,part_pk,res['filename'])
    print('done with getting imagefilename list')

def move_part(doc_pk,part_pk,place):
    print('move part_pk '+str(part_pk)+'to place '+str(place))
    url=root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+'/move/'
    data='{"index":'+str(place)+'}'
    res=requests.post(url,headers=headers,data=data)
    print(res)

def repolygonize(doc_pk,part_pk):
    print('repolygonize : ',part_pk)
    parts_url=root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)
    url_repoly=urljoin(parts_url,'/reset_masks/')
    repoly_res=requests.post(url_repoly,headers=headers)
    print('done repolygonizing')

def repolygonize_line(doc_pk,part_pk,line_pk):
    print('repolygonize line : ',line_pk)
    parts_url=root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)
    url_repoly=parts_url+'/reset_masks/?only='+str(line_pk)
    repoly_res=requests.post(url_repoly,headers=headers)
    print('done repolygonizing')
    return repoly_res

def reorder(doc_pk,part_pk):
    print('reorder ',part_pk)
    reorder_url=root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+'/recalculate_ordering/'
    reorder_res = requests.post(reorder_url,headers=headers)
    print('done reordering')

def check_point_in_image(p,imgHeight,imgWidth,safetydistance):
    # deal with x:
    p[0]=min(imgWidth-safetydistance,max(p[0],safetydistance))
    p[1]=min(imgHeight-safetydistance,max(p[1],safetydistance))
    return p

def get_centroid(line):
    return [line[0,0]+line[-1][0],line[0,1]+line[-1][1]]

def get_angle(line):
    return rad2deg(arctan2(line[-1][-1] - line[0][-1], line[-1][0] - line[0][0]))

def get_vector(angle,length):
  #from numpy import rad2deg,deg2rad,cos,sin
  #return rad2deg(arctan2(line[-1][-1] - line[0][-1], line[-1][0] - line[0][0]))
  #length=hypothenuse
  #ankathete=x
  #gegenkathete=y
  y=int(round(sin(deg2rad(angle))*length,0))
  x=int(round(cos(deg2rad(angle))*length,0))
  return x,y

def check_baseline(doc_pk,part_pk,mindist):
  res = requests.get(url='https://www.escriptorium.fr/api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+'/',headers=headers).json()
  imgWidth,imgHeight = res['image']['size']
  lines=res['lines']
  change = False
  for n,line in enumerate(lines):
    baselinepoints = line['baseline']
    for point in baselinepoints:
      if point[0] < 1:
        change = True
        print(part_pk,n,'x', point[0])
        res['lines'][n]['lines'][n][0] = mindist
      if point[0] >= imgWidth:
        change = True
        print(part_pk,n,'x', point[0])
        res['lines'][n][0] = imgWidth-mindist
      if point[1] < 1:
        change = True
        print(part_pk,n,'y', point[1])
        res['lines'][n][1] = mindist
      if point[1] >= imgHeight:
        change = True
        print(part_pk,n,'y', point[1])
        res['lines'][n][1] = imgHeight-mindist
  return res


def rotateNP(p, origin=(0, 0), degrees=0):
    angle = deg2rad(degrees)
    R = array([[cos(angle), -sin(angle)],
                  [sin(angle),  cos(angle)]])
    o = atleast_2d(origin)
    p = atleast_2d(p)
    return squeeze((R @ (p.T-o.T) + o.T).T)


def line_intersection(line1, line2):
    xdiff = (line1[0][0] - line1[1][0], line2[0][0] - line2[1][0])
    ydiff = (line1[0][1] - line1[1][1], line2[0][1] - line2[1][1])

    def det(a, b):
        return a[0] * b[1] - a[1] * b[0]

    div = det(xdiff, ydiff)
    if div == 0:
        print('lines do not intersect')
        x = line1[0][0]
        y = 100000
    else:
        d = (det(*line1), det(*line2))
        x = det(d, xdiff) / div
        y = det(d, ydiff) / div
    return x, y

def rename_tr_level(doc_pk,tr_level,new_name):
  url=root_url+'api/documents/'+str(doc_pk)+'/transcriptions/'+str(tr_level)+'/'
  rjson=requests.get(url,headers=headers).json()
  rjson['name']=new_name
  r=requests.put(url,headers=headers,data=json.dumps(rjson))
  return r

def simplify_line_polygons_on_part(doc_pk,part_pk,simplification=10):
  lines_url=root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+'/lines/'
  lines=loop_through_itempages_and_get_all(lines_url)
  for line in lines:
    mask=line['mask']
    #print(len(mask),mask)
    maskPoly=Polygon(mask)
    maskSimple=maskPoly.simplify(10)
    #print(maskSimple)
    x,y = maskSimple.exterior.coords.xy
    new_mask = [list(x) for x in zip(x,y)]
    #print(len(new_mask),new_mask)
    line_url = lines_url+str(line['pk'])+'/'
    line_json={'mask' : new_mask}
    res_line = requests.patch(line_url,headers=headers,data=json.dumps(line_json))

#if __name__ == '__main__':
  #  print("<> Running the module as a script! <>")
    #get_transcription_levels(doc_pk)


In [ ]:
#@title complex
#%%writefile complex_api_functions.py

# find_and_replace_text_or_trim, transfer_txt2level, get_pages_without_transcription_level, get_pages_with_wordstring_in_transcription_level,
# delete_linefree_regions, keep_only_biggest_region, associate_lines_with_existing_regions, delete_unlinked_lines, delete_1p_lines,
# find_lines_without_mask, extend_lines, create_transcription_table, delete_empty_lines

import json
import requests
import os
from requests.compat import urljoin
from shapely.geometry import Point, Polygon
#from basic_api_functions import *
from numpy import rad2deg,deg2rad,array,atleast_2d,cos,sin,arctan2,squeeze,cross,linalg
from statistics import mean
from PIL import Image
from time import sleep

def find_and_replace_text_or_trim(doc_pk,part_pk,tr_level,chars,replacement_char,do_replace,do_trim):
  return_part_pk=''
  if do_replace or do_trim:
    if do_replace:
      print('replacing '+chars+' with '+replacement_char)
    if do_trim:
      print('trim')
  else:
    print('find pages with '+chars)
    chars=set(chars)
  page_nu=0
  while True:
    page_nu+=1
    url=root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+'/transcriptions/'+'?page='+str(page_nu)+'&transcription='+str(tr_level)
    #print('checking: '+root_url+'document/'+str(doc_pk)+'/part/'+str(part_pk)+'/edit/')
    res_transcriptions_this_part = requests.get(url,headers=headers).json()
    #print(res_transcriptions_this_part)
    if not(do_replace or do_trim):
      for m,line in enumerate(res_transcriptions_this_part['results']):
        linestr=line['content']
        if any((c in chars) for c in linestr):
          print('have a look for character in ',part_pk,' line :',m)
          return_part_pk=part_pk
    else:
      for m,line in enumerate(res_transcriptions_this_part['results']):
        oldstr=line['content']
        if ((do_replace and any((c in chars) for c in oldstr)) or do_trim):
          t_url=(root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+'/transcriptions/'+str(line['pk'])+'/')
          res_t=requests.get(t_url,headers=headers).json()
          content=res_t['content']
          #print('old:',content)
          content=content.replace(chars,replacement_char)
          #print('new:',content)
          if do_trim:
            content=content.strip()
          if (do_replace or do_trim) and not(oldstr==content):
            res_t['content']=content
            data=json.dumps(res_t, ensure_ascii=False).encode('utf8')
            res_put=requests.put(t_url,headers=headers,data=data)
    if not(res_transcriptions_this_part['next']):
      break
  return return_part_pk

def transfer_txt2level(doc_pk,part_pk,source_tr_level,target_tr_level):
  target_tr_url=root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+'/transcriptions/'
  page_nu=0
  while True:
    page_nu+=1
    source_tr_url=root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+'/transcriptions/'+'?page='+str(page_nu)+'&transcription='+str(source_tr_level)
    #print(source_tr_url)
    res = requests.get(source_tr_url, headers=headers).json()
    for line in res['results']:
      line_pk=line['line']
      line_url=root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+'/lines/'+str(line_pk)
      #print(line_url)
      res_line=requests.get(line_url,headers=headers).json()
      #print(res_line)
      content=line['content']
      if not(res_line['typology']==3):
         data = {'line': line_pk, 'transcription': target_tr_level, 'content': content}
         res_post = requests.post(target_tr_url, data=json.dumps(data), headers=headers)
    if not(res['next']):
      break

def transfer_txt2level_overwrite(doc_pk,part_pk,source_tr_level,target_tr_level):
  page_nu=0
  while True:
    page_nu+=1
    source_tr_url=root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+'/transcriptions/'+'?page='+str(page_nu)+'&transcription='+str(source_tr_level)
    print(source_tr_url)
    res1 = requests.get(source_tr_url, headers=headers).json()
    target_tr_url=root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+'/transcriptions/'+'?page='+str(page_nu)+'&transcription='+str(target_tr_level)
    res2 = requests.get(target_tr_url, headers=headers).json()
    tg_line_pk_list = [line.get('line') for line in res2['results']]
    for line in res1['results']:
      line_pk=line['line']
      content=line['content']
      #print(line)
      try:
        tg_line_pk=res2['results'][tg_line_pk_list.index(line_pk)]['pk']
        data = json.dumps({'line': line_pk, 'transcription': target_tr_level, 'content': content})
        target_tr_url=root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+'/transcriptions/'+str(tg_line_pk)+'/'
        res_tg = requests.put(target_tr_url, data=data, headers=headers)
      except:
        print('not found: ', line_pk, '--> posted')
        data = json.dumps({'line': line_pk, 'transcription': target_tr_level, 'content': content})
        target_tr_url=root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+'/transcriptions/'
        res_tg = requests.post(target_tr_url, data=data, headers=headers)
      #print(res_tg)
    if not(res1['next']):
      break

def get_lines_without_transcription_level(doc_pk,part_pk,tr_level,also_empty_lines=False):
  print('check pages without transcription in transcription level for pk', part_pk)
  url=root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+'/transcriptions/?transcription='+str(tr_level)
  res_transcriptions_this_part = requests.get(url,headers=headers).json()
  url=root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)
  res_part = requests.get(url,headers=headers).json()
  line_pk_list_this_part = [line.get('pk') for line in res_part['lines']]
  line_pk_list_this_part_set=set(line_pk_list_this_part)
  line_pk_list_this_transcription = [tr.get('line') for tr in res_transcriptions_this_part['results']]
  line_pk_list_this_transcription_set = set(line_pk_list_this_transcription)
  missing=line_pk_list_this_part_set.difference(line_pk_list_this_transcription_set)
  if len(line_pk_list_this_part)>0:
    proportion=len(missing)/len(line_pk_list_this_part)
  else:
    proportion=0
  empty_lines=[]
  if also_empty_lines:
    for line in res_transcriptions_this_part['results']:
      if line['content']=='':
        empty_lines.append(line['pk'])
  return missing,proportion,empty_lines

def get_pages_with_wordstring_in_transcription_level(doc_pk,part_list,tr_level,words):
## check which pages contain any given string in a specific transcription level part 2
  print('check pages with string in transcription level')
  for n,part_pk in enumerate(part_list):
    if n % 50 == 0:
      print(n)
    transcriptions_this_part=get_page_transcription(doc_pk,part_pk,tr_level)
    for m,line in enumerate(transcriptions_this_part):
      linewords=line.split()
      #print(linewords)
      if any((w in words) for w in linewords):
        print('check part '+str(n+1)+' line: '+str(m+1))
        oldstr=line
        print(oldstr)
        print(root_url+'document/'+str(doc_pk)+'/part/'+str(part_pk)+'/edit/')
  print('done')

def find_or_delete_linefree_regions(doc_pk,part_pk,do_delete=False):

  parts_url = urljoin(root_url,'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+'/')
  print(parts_url)
  res = requests.get(parts_url,headers=headers).json()
  inhabited_region_list=set(get_inhabited_region_pk_list(doc_pk,part_pk))
  region_list=set(get_region_pk_list(doc_pk,part_pk))
  uninhabited_region_list=region_list.difference(inhabited_region_list)
  for region in uninhabited_region_list:
    print('found linefree region')
    if do_delete:
      delete_url=root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+ '/blocks/'+str(region)
      r = requests.delete(delete_url,headers=headers)

def keep_only_biggest_region(doc_pk,part_pk):

  region_list = get_region_pk_list(doc_pk,part_pk)
  biggest_area=0
  biggest_area_pk=0
  biggest_area_n=0
  all_regions=[]
  for m,region_nu in enumerate(region_list):
    part_block_url = root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+ '/blocks/'+str(region_nu)+'/'
    region = requests.get(part_block_url,headers=headers).json()
    all_regions.append(region)
    region_polygon=Polygon(region['box'])
    if biggest_area<region_polygon.area:
      biggest_area=region_polygon.area
      biggest_area_pk=region['pk']
      keep_region=region
      keep_region['order']=0
  part_block_url =root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+ '/blocks/?format=json'
  for n,region in enumerate(all_regions):
    if not(region['pk']==biggest_area_pk):
      print('delete '+str(region['pk']))
      delete_url=root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+ '/blocks/'+str(region['pk']) + '/'
      r = requests.delete(delete_url,headers=headers)
      print(r.status_code)

def associate_lines_with_existing_regions(doc_pk,part_pk):
  print(part_pk)
  line_url=root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+ '/lines/'
  region_url=root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+ '/blocks/'
  lines=get_all_item_info(line_url)
  regions=get_all_item_info(region_url)
  nu_regions=len(regions)
  region_pk_list = [item['pk'] for item in regions]
  region_polygons = [item['box'] for item in regions]
  line_pk_list = [item['pk'] for item in lines]
  baseline_coords_list = [item['baseline'] for item in lines]
  print('associating')
  for n,line_pk in enumerate(line_pk_list):
    baseline=baseline_coords_list[n]
    centroidx=int(round((baseline[0][0]+baseline[-1][0])/2))
    centroidy=int(round((baseline[0][1]+baseline[-1][1])/2))
    p = Point(centroidx, centroidy)
    region_nu = 0
    while (region_nu<nu_regions):
      poly = Polygon(region_polygons[region_nu])
      if p.within(poly):
        # line_url = root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+ '/lines/' + str(line_pk) + '/'
        # line_json={'region' : region_pk_list[region_nu]}
        # res_line = requests.patch(line_url,headers=headers,data=json.dumps(line_json))
        lines[n]['region']=region_pk_list[region_nu]
        break
      else:
        region_nu+=1
  print('putting')
  bulk_json=json.dumps({'lines': lines})
  #print(bulk_json[0:300])
  bulk_url=root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+'/lines/bulk_update/'
  #print(bulk_url)
  res_bulk=requests.put(bulk_url,headers=headers,data=bulk_json)
  print(res_bulk)
  print('reordering')
  reorder_url=root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+'/recalculate_ordering/'
  reorder_res = requests.post(reorder_url,headers=headers)


def find_or_delete_unlinked_lines(doc_pk,part_pk,do_delete=False):
  parts_url = urljoin(root_url,'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+'/')
  print(parts_url)
  res = requests.get(parts_url,headers=headers).json()
  for line_n,line in enumerate(res['lines']):
    if line['region']==None:
      print('found unassociated line')
      if do_delete:
        delete_url=root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+ '/lines/'+str(line['pk'])
        r = requests.delete(delete_url,headers=headers)
  if do_delete:
    reorder_url=root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+'/recalculate_ordering/'
    reorder_res = requests.post(reorder_url,headers=headers)

def delete_1p_lines(doc_pk,part_pk):

  parts_url = urljoin(root_url,'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+'/')
  print(parts_url)
  res = requests.get(parts_url,headers=headers).json()
  #print(res)

  for line_n,line in enumerate(res['lines']):
    if (len(line['baseline'])<2):
      print('need to delete line')
      delete_url=root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+ '/lines/'+str(line['pk'])
      r = requests.delete(delete_url,headers=headers)
  reorder_url=root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+'/recalculate_ordering/'
  reorder_res = requests.post(reorder_url,headers=headers)

def find_lines_without_mask(doc_pk,part_pk):

  from requests.compat import urljoin
  parts_url = urljoin(root_url,'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+'/')
  res = requests.get(parts_url,headers=headers).json()
  for line_n,line in enumerate(res['lines']):
    try:
      len_mask=len(line['mask'])
    except:
      len_mask=0
    if len_mask<4:
      print(line_n,len_mask)

def extend_lines(doc_pk,part_pk,extension=15,left_also=False,baseline2topline=False,repoly=True):

  parts_url = urljoin(root_url,'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+'/')
  res = requests.get(parts_url,headers=headers).json()
  pagewidth=res['image']['size'][0]
  for line in res['lines']:
    baseline=line['baseline']
    line_id=line['pk']
    if (line['document_part']==part_pk):
      if baseline[0][0]<baseline[-1][0]:
        if left_also:
          baseline[0][0]=max(5,int(round((baseline[0][0])))-extension)
        baseline[-1][0]=min(int(round((baseline[-1][0])))+extension,pagewidth-5)
      else:
        if left_also:
          baseline[0][0]=min(int(round((baseline[0][0])))+extension,pagewidth-5)
        baseline[-1][0]=max(5,int(round((baseline[-1][0])))-extension)
      if baseline2topline:
        baseline=[[pt[0], max(5,pt[1]-y_decrease)] for pt in baseline]
      line_url = root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+ '/lines/' + str(line_id) + '/'
      line_json = requests.get(line_url,headers=headers).json()
      line_json['baseline']=baseline
      res_line = requests.put(line_url,headers=headers,data=json.dumps(line_json))
  if repoly:
    url_repoly=urljoin(parts_url,'reset_masks/')
    repoly_res=requests.post(url_repoly,headers=headers)

def create_transcription_table(doc_pk,part_list,tr_level):

  pages=[]
  regions=[]
  line_pks=[]
  transcription_pks=[]
  linenumbers=[]
  txt=[]
  regions=[]
  for n,part_pk in enumerate(part_list):

    if n % 10 == 0:
      print(n)
    url=root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+'/transcriptions/?transcription='+str(tr_level)
    res_transcriptions_this_part = requests.get(url,headers=headers).json()
    line_pk_list_this_transcription = [tr.get('line') for tr in res_transcriptions_this_part['results']]

    url=root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)
    res_part = requests.get(url,headers=headers).json()
    line_pk_list_this_part = [line.get('pk') for line in res_part['lines']]
    line2region_list_this_part = [line.get('region') for line in res_part['lines']]

    region_pk_list_this_part = [region.get('pk') for region in res_part['regions']]

    for line in res_part['lines']:
      pages.append(n+1)
      line_pks.append(line['pk'])
      linenumbers.append(line['order']+1)
      try:
        regions.append(region_pk_list_this_part.index(line['region'])+1)
      except:
        regions.append(None)
      try:
        index=line_pk_list_this_transcription.index(line['pk'])
        transcription_pks.append(res_transcriptions_this_part['results'][index]['pk'])
        txt.append(res_transcriptions_this_part['results'][index]['content'])
      except:
        transcription_pks.append(None)
        txt.append('')
  print('done')

  columntitles = ['page', 'region', 'line','line_pk', 'transcription_pk','txt']
  data =[columntitles] + list(zip(pages, regions, linenumbers,line_pks,transcription_pks,txt))

  for i, d in enumerate(data):
    tableline = ' | '.join(str(x).ljust(4) for x in d)
    print(tableline)
    if i == 0:
      print('-' * len(tableline))

def delete_empty_lines(doc_pk,part_pk,tr_level,min_length=1):

  url=root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_nu)+'/transcriptions/?transcription='+str(tr_level)
  print(root_url+'document/'+str(doc_pk)+'/part/'+str(part_nu)+'/edit#')
  res_transcriptions_this_part = requests.get(url,headers=headers).json()
  line_pk_list_this_transcription = [tr.get('line') for tr in res_transcriptions_this_part['results']]

  url=root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_nu)
  res_part = requests.get(url,headers=headers).json()
  line_pk_list_this_part = [line.get('pk') for line in res_part['lines']]
  line2region_list_this_part = [line.get('region') for line in res_part['lines']]
  region_pk_list_this_part = [region.get('pk') for region in res_part['regions']]

  for line in res_part['lines']:
    try:
      index=line_pk_list_this_transcription.index(line['pk'])
      txt=res_transcriptions_this_part['results'][index]['content']
      if (txt=='') or (txt==' ') or len(txt)<min_length:
        delete_url=url=root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_nu)+'/lines/'+str(line['pk'])
        del_res=requests.delete(delete_url, headers=headers)
        print(del_res.status_code)
        print(delete_url)
    except:
      delete_url=url=root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_nu)+'/lines/'+str(line['pk'])
      del_res=requests.delete(delete_url, headers=headers)
      print(del_res.status_code)

def calculate_average_line_distance(baselineCoordsList,pageheight,pagewidth):
  average_line_height_list=[]
  midpointList = [[(line[0][0]+line[-1][0])/2,(line[0][1]+line[-1][1])/2] for line in baselineCoordsList]
  lineBegList = [line_intersection([[0,0],[0,pageheight]],[[line[0][0],line[0][1]],[line[-1][0],line[-1][1]]]) for line in baselineCoordsList]
  lineEndList = [line_intersection([[pagewidth,0],[pagewidth,pageheight]],[[line[0][0],line[0][1]],[line[-1][0],line[-1][1]]]) for line in baselineCoordsList]
  d_list=[]
  for n,midpoint in enumerate(midpointList):
    if n>0:
      p1=array(lineBegList[n-1])
      p2=array(lineEndList[n-1])
      p3=array(midpoint)
      d_list.append(cross(p2-p1,p3-p1)/linalg.norm(p2-p1))
    elif len(midpointList)>1:
      p1=array(lineBegList[n+1])
      p2=array(lineEndList[n+1])
      p3=array(midpoint)
      d_list.append(-(cross(p2-p1,p3-p1)/linalg.norm(p2-p1)))
    average_line_height=int(round(sum(d_list)/len(d_list),0))
    average_line_height_list.append(average_line_height)
  if average_line_height_list:
    return int(round(mean(average_line_height_list)))
  else:
    return -1

def restrict_first_and_last_line_polygon_according_to_average_line_height(doc_pk,part_pk,region_pk,average_line_distance=0):

  def repoly_extreme_line(line,direction):
    lines_url = root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+'/lines/'
    # create dummy baseline above or below to limit extension of first or last line
    phantombaseline=[[pt[0], min(pageheight-5,max(5,pt[1]+(average_line_distance+5)*direction))] for pt in line[2]]
    phantom_data=json.dumps({'document_part':part_pk,'region':region_pk,'baseline':phantombaseline})
    phantom_line_pk=requests.post(lines_url,headers=headers,data=phantom_data).json()['pk']
    # repolygonize
    res=repolygonize_line(doc_pk,part_pk,line[0])
    print('repoly-response: ',res)
    # delete dummy line
    phantom_line_del=requests.delete(lines_url+str(phantom_line_pk),headers=headers)
  print(average_line_distance)
  pageheight,pagewidth=page_height_width(doc_pk,part_pk)
  all_lines_this_region=get_all_lines_of_region(doc_pk,part_pk,region_pk)
# main lines only
  main_lines_this_region =[line for line in all_lines_this_region if line['typology'==None]]
  baselineCoordsList=[line[2] for line in main_lines_this_region]
# calculate average line distance
  if average_line_distance==0:
    average_line_distance=calculate_average_line_distance(baselineCoordsList,pageheight,pagewidth)
  if (average_line_distance>5) and (len(main_lines_this_region)>0):
# treat first line
    index_first = min(lines[1] for lines in main_lines_this_region)
    line_first=[line for line in main_lines_this_region if line[1]==index_first]
    line_first=line_first[0]
    repoly_extreme_line(line_first,-1)
# treat last line
    index_last = max(lines[1] for lines in main_lines_this_region)
    if not(index_last==index_first):
      line_last=[line for line in main_lines_this_region if line[1]==index_last]
      line_last=line_last[0]
      repoly_extreme_line(line_last,1)
      print('repolygonized extremes of part: ',part_pk)
  else:
    print('not able to calculate average line distance for part: ',part_pk)
  return average_line_distance

def create_normalized_polygons_around_CORRECTION_lines(doc_pk,part_pk,ascender=12,descender=24,correctionlinetype=3,safetydistance=5):

  pageheight,pagewidth=page_height_width(doc_pk,part_pk)
  ## first loop: calculate average line height and distance
  region_list=get_main_region_pk_list(doc_pk,part_pk)
  for m,region_pk in enumerate(region_list):
    all_lines_this_region=get_all_lines_of_region(doc_pk,part_pk,region_pk)
    correction_lines_this_region = [line for line in all_lines_this_region if line[3]==correctionlinetype]
    baselineCoordsList = [line[2] for line in correction_lines_this_region]
    baselinePKList = [line[0] for line in correction_lines_this_region]
    for n,line in enumerate(baselineCoordsList):
      old_pt=line[0]
      for o,pt in enumerate(line[1:]):
        #print('region '+str(m)+', line '+str(n)+', point '+str(o)+str(old_pt)+':'+str(pt))
        angle=get_angle([old_pt,pt])
        if o==0:
          inside_above=False
          inside_below=False
          new_pt_above=rotateNP((old_pt[0],old_pt[1]-ascender),origin=tuple(old_pt),degrees=angle)
          new_pt_below=rotateNP((old_pt[0],old_pt[1]+descender),origin=tuple(old_pt),degrees=angle)
          #print(new_pt_above,pageheight,pagewidth,safetydistance)
          new_pt_above=check_point_in_image(new_pt_above,pageheight,pagewidth,safetydistance)
          new_pt_below=check_point_in_image(new_pt_below,pageheight,pagewidth,safetydistance)
          #print(new_pt_above)
          polygon_above=[[int(coordinate) for coordinate in new_pt_above]]
          polygon_below=[[int(coordinate) for coordinate in new_pt_below]]
        new_pt_above=[int(coordinate) for coordinate in rotateNP((pt[0],pt[1]-ascender),origin=tuple(pt),degrees=angle)]
        new_pt_below=[int(coordinate) for coordinate in rotateNP((pt[0],pt[1]+descender),origin=tuple(pt),degrees=angle)]
        new_pt_above=check_point_in_image(new_pt_above,pageheight,pagewidth,safetydistance)
        new_pt_below=check_point_in_image(new_pt_below,pageheight,pagewidth,safetydistance)
        if o>0:
          if inside_above and not(o==len(line[1:])): # if point is inside and is not the last point, # delete previous point, dont insert next point either # and insert instead the intersection of (p-2:p-1) and (p:p+1)
            replace_pt=list(line_intersection([polygon_above[-3],polygon_above[-2]],[polygon_above[-1],new_pt_above]))
            del polygon_above[-2:-1]
            polygon_above.append(replace_pt)
          if inside_below:
            replace_pt=list(line_intersection([polygon_below[2],polygon_below[1]],[polygon_below[0],new_pt_below]))
            del polygon_below[0:1]
            polygon_below.insert(0,replace_pt)
          pol=Polygon(polygon_above+polygon_below)
          inside_above=pol.contains(Point(new_pt_above))
          inside_below=pol.contains(Point(new_pt_below))
        polygon_above.append(new_pt_above) # if point is outside add it to boundary
        polygon_below.insert(0,new_pt_below) # if point is outside add it to boundary
        old_pt=pt
      if inside_above:
        del polygon_above[-1] # if point is inside and is last point delete it
      if inside_below:
        del polygon_below[0] # if point is inside and is last point delete it
      polygon_boundary=polygon_above+polygon_below
      line_url = urljoin(root_url,'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+'/lines/'+str(baselinePKList[n])+'/')
      resline = requests.get(line_url,headers=headers).json()
      resline['mask']=polygon_boundary
      data=json.dumps(resline)
      rput = requests.put(line_url,headers=headers,data=data)
      print(m,rput)


def export_txt_file_of_main_text_in_main_regions(doc_nu,part_list,tr_level_pk,dirname,fname):

    #os.chdir(dirname)
    #file = open(fname, 'w+',encoding="utf8")
    resultstr=''
    for n,part_pk in enumerate(part_list):
        #print(n,part_pk)
        #get all main regions
        page_nu=0
        main_region_list=[]
        while True:
            page_nu+=1
            regions_url = root_url+'api/documents/'+str(doc_nu)+'/parts/'+str(part_pk)+'/blocks/'+'?page='+str(page_nu)
         #   print(regions_url)
            regions=requests.get(regions_url,headers=headers).json()
            main_region_list=main_region_list+[r.get('pk') for r in regions['results'] if r['typology']==2]
            if not(regions['next']):
                break
        #get all transcriptions of this part
        transcriptions={}
        page_nu=0
        while True:
            page_nu+=1
            tr_url=root_url+'api/documents/'+str(doc_nu)+'/parts/'+str(part_pk)+'/transcriptions/'+'?page='+str(page_nu)+'&transcription='+str(tr_level_pk)
            res = requests.get(tr_url, headers=headers).json()
            for tr_line in enumerate(res['results']):
                transcriptions[tr_line[1]['line']]=tr_line[1]['content']
            if not(res['next']):
                break

        #get all lines of this part that are in the main columns and of the default type
        page_nu=0
        while True:
            page_nu+=1
            lines_url=root_url+'api/documents/'+str(doc_nu)+'/parts/'+str(part_pk)+'/lines/'+'?page='+str(page_nu)
            res = requests.get(lines_url, headers=headers).json()
            for m,line in enumerate(res['results']):
                if (line['typology']==None) and (line['region'] in main_region_list):
                    try:
                        content=transcriptions[line['pk']]
                    except:
                        content=''
                    table_line=str(n+1)+'\t'+str(part_pk)+'\t'+str(line['order'])+'\t'+str(line['pk'])+'\t'+content+'\n'
                    resultstr=resultstr+table_line
     #               file.write(table_line)
                    print(table_line)
            if not(res['next']):
                break
    #file.close()
    return resultstr
    print('done')



def simplify_hebrew(doc_nu,part_list,source_tr_level,target_tr_level):
    vocalization=[1425,1427,1430,1436,1438,1443,1446,1453,1456,1457,1458,1459,1460,1461,1462,1463,1464,1465,1466,1467,1468,1469,1471,1473,1474,1476,1477,1478,1479]
    singlequot=[1523,8216,8217,8219,8242]
    doublequot=[1524,8220,8221,8222,8223,8243]
    spacechars=[9,160,8201,8195,8192]
    deletechars=[1565,8299,8205,8300,8302]
    toreplacechars=[(64296,1514),(64298,1513),(64299,1513),(64300,1513),(64302,1488),(64303,1488),(64305,1489),(64306,1490),(64307,1491),(64309,1493),(64315,1499),(64324,1508),(64327,1511),(64330,1514),(64331,1493),(64332,1489),(64333,1499),(64334,1508),(8277,42),(8283,46),(11799,61)]
    for n,part in enumerate(part_list[41:]):
        target_tr_url=root_url+'api/documents/'+str(doc_nu)+'/parts/'+str(part)+'/transcriptions/'
        #https://www.escriptorium.fr/api/documents/39/parts/14480/transcriptions/?page=2&transcription=46
        page_nu=0
        while True:
            page_nu+=1
            source_tr_url=root_url+'api/documents/'+str(doc_nu)+'/parts/'+str(part)+'/transcriptions/'+'?page='+str(page_nu)+'&transcription='+str(source_tr_level)
            target_tr_url=root_url+'api/documents/'+str(doc_nu)+'/parts/'+str(part)+'/transcriptions/'+'?page='+str(page_nu)+'&transcription='+str(target_tr_level)
            res = requests.get(source_tr_url, headers=headers).json()
            print(source_tr_url)
            res_target=requests.get(target_tr_url, headers=headers).json()
            if res_target!={'detail': 'Invalid page.'}:
                target_line_pk_list = [line['line'] for line in res_target['results']]
                target_tr_pk_list = [line['pk'] for line in res_target['results']]
                print(target_line_pk_list)
            else:
                target_line_pk_list=[]
                target_tr_pk_list=[]
            #print(res_target)
            for m,line in enumerate(res['results']):
                #'line': 674381, 'transcription': 46, 'content': 'מדביק מרחם שערים הליכות מצויינים',
                line_pk=line['line']
                print(line_pk)
                print(line['pk'])
                content=line['content']
                for u in vocalization:
                    content=content.replace(chr(u),'')
                for u in spacechars:
                    content=content.replace(chr(u),' ')
                for u in deletechars:
                    content=content.replace(chr(u),'')
                for u in singlequot:
                    content=content.replace(chr(u),"'")
                for u in doublequot:
                    content=content.replace(chr(u),'"')
                for (x,y) in toreplacechars:
                    content=content.replace(chr(x),chr(y))
                data = {'line': line_pk, 'transcription': target_tr_level, 'content': content}
                if line_pk in target_line_pk_list:
                    t_url=(root_url+'api/documents/'+str(doc_nu)+'/parts/'+str(part)+'/transcriptions/'+str(line['pk'])+'/')
                    res_t=requests.get(t_url,headers=headers).json()
                    print('put',res_t['content'])
                    res_t['content']=content
                    print(t_url)
                    data=json.dumps(res_t, ensure_ascii=False).encode('utf8')
                    res_put = requests.put(t_url, data=data, headers=headers)
                    #print('put',res_put.content)
                else:
                    res_post = requests.post(target_tr_url, data=json.dumps(data), headers=headers)
                    #print(n,part,m,res_post.content)
            if not(res['next']):
                break
    print('done simplifying hebrew')

def delete_text_in_main_regions_except_line_type(doc_nu,part_list,tr_level,allowed_line_types):

    for n,part_pk in enumerate(part_list):
        print(n,part_pk)
        #get all main regions
        regions_url = root_url+'api/documents/'+str(doc_nu)+'/parts/'+str(part_pk)+'/blocks/'
        regions=requests.get(regions_url,headers=headers).json()
        main_region_list=[r.get('pk') for r in regions['results'] if r['typology']==2]

        #get all lines of this part that are in the main columns and of the default type and delete their content
        page_nu=0
        del_lines_pk_list=[]
        while True:
            page_nu+=1
            lines_url=root_url+'api/documents/'+str(doc_nu)+'/parts/'+str(part_pk)+'/lines/'+'?page='+str(page_nu)
            res = requests.get(lines_url, headers=headers).json()
            for line in res['results']:
                if (not(line['typology'] in allowed_line_types)) and (line['region'] in main_region_list):
                    del_lines_pk_list.append(line['pk'])
            if not(res['next']):
                break

        page_nu=0
        while True:
            page_nu+=1
            tr_url=root_url+'api/documents/'+str(doc_nu)+'/parts/'+str(part_pk)+'/transcriptions/'+'?page='+str(page_nu)+'&transcription='+str(tr_level)
            res = requests.get(tr_url, headers=headers).json()
            print(del_lines_pk_list)
            for tr_line in res['results']:
                if (tr_line['line'] in del_lines_pk_list) and not(tr_line['content']==''):
                    tr_url=root_url+'api/documents/'+str(doc_nu)+'/parts/'+str(part_pk)+'/transcriptions/'+str(tr_line['pk'])+'/'
                    print(tr_url)
                    data = {'line': tr_line['line'], 'transcription': tr_level, 'content': ''}
                    resput = requests.put(tr_url, data=json.dumps(data), headers=headers)
                    print(resput)
            if not(res['next']):
                break
    print('done')

def restrict_first_and_last_line_polygon_according_to_average_line_height(doc_pk,part_pk,region_pk,average_line_distance=0):

  def repoly_extreme_line(line,direction):
    lines_url = root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+'/lines/'
    # create dummy baseline above or below to limit extension of first or last line
    #single line: direction=0
    phantom_line_pks=[]
    if not(direction==0): #NOT a single-line-region (direction=0) but a top or bottom line of multiline region
      phantombaseline=[[pt[0], min(pageheight-5,max(5,pt[1]+(average_line_distance+5)*direction))] for pt in line[2]]
      phantom_data=json.dumps({'document_part':part_pk,'region':region_pk,'baseline':phantombaseline})
      phantom_line_pks.append(requests.post(lines_url,headers=headers,data=phantom_data).json()['pk'])
    else: # if region consists of single line create dummy lines above AND below
      for direction in [-1,1]:
        phantombaseline=[[pt[0], min(pageheight-5,max(5,pt[1]+(average_line_distance+5)*direction))] for pt in line[2]]
        phantom_data=json.dumps({'document_part':part_pk,'region':region_pk,'baseline':phantombaseline})
        phantom_line_pks.append(requests.post(lines_url,headers=headers,data=phantom_data).json()['pk'])
    # repolygonize
    res=repolygonize_line(doc_pk,part_pk,line[0])
    print('repoly-response: ',res)
    sleep(2)
    # delete dummy line(s)
    for phantom_line_pk in phantom_line_pks:
      phantom_line_del=requests.delete(lines_url+str(phantom_line_pk),headers=headers)
  print(average_line_distance)
  pageheight,pagewidth=page_height_width(doc_pk,part_pk)
  all_lines_this_region=get_all_lines_of_region(doc_pk,part_pk,region_pk)
# main lines only
  main_lines_this_region =[line for line in all_lines_this_region if line['typology'==None]]
  baselineCoordsList=[line[2] for line in main_lines_this_region]
# calculate average line distance
  if average_line_distance==0:
    average_line_distance=calculate_average_line_distance(baselineCoordsList,pageheight,pagewidth)
  if (average_line_distance>5) and (len(main_lines_this_region)>0):
# treat first line
    index_first = min(lines[1] for lines in main_lines_this_region)
    line_first=[line for line in main_lines_this_region if line[1]==index_first]
    line_first=line_first[0]
    index_last = max(lines[1] for lines in main_lines_this_region)
# treat last line
    if not(index_last==index_first):
      repoly_extreme_line(line_first,-1)
      line_last=[line for line in main_lines_this_region if line[1]==index_last]
      line_last=line_last[0]
      repoly_extreme_line(line_last,1)
    else:
      repoly_extreme_line(line_first,0) #direction 0 == single line region
    print('repolygonized extremes of part: ',part_pk)
  else:
    print('not able to calculate average line distance for part: ',part_pk)
  return average_line_distance


def loop_document_restrict_extreme_lines_all_regions(doc_pk,fix_line_height,start_item=0):

  part_pk_list,tr_level_list,region_type_list,line_type_list=get_basic_info(doc_pk)
  for part_pk in part_pk_list[start_item:]:
    print('----------------------------------------------------')
    print('resegmenting part :',part_pk)
    region_list=get_region_pk_list(doc_pk,part_pk)
    main_region_list=get_main_region_pk_list(doc_pk,part_pk)
    average_line_height_in_main_regions=[]
    for region_pk in main_region_list:
      average_line_height_in_main_regions.append(restrict_first_and_last_line_polygon_according_to_average_line_height(doc_pk,part_pk,region_pk,fix_line_height))
    if len(main_region_list)>0:
      global_average_line_height=mean(average_line_height_in_main_regions)
    else:
      global_average_line_height=fix_line_height
    non_main_regions=set(region_list)-set(main_region_list)
    for region_pk in non_main_regions:
      line_height=restrict_first_and_last_line_polygon_according_to_average_line_height(doc_pk,part_pk,region_pk,global_average_line_height)

  print('done')

def create_normalized_polygons_around_lines(doc_nu,splitfactor=0.6,safetydistance=5,basic_line_height=120):

    part_list,tr_level_list,region_type_list,line_type_list=get_basic_info(doc_nu)

    for n,part in enumerate(part_list):
        parts_url = urljoin(root_url,'api/documents/'+str(doc_nu)+'/parts/'+str(part)+'/')
        res = requests.get(parts_url,headers=headers).json()
        pagewidth=res['image']['size'][0]
        pageheight=res['image']['size'][1]
        print(parts_url)
        ## first loop: calculate average line height and distance
        regionList = [region for region in res['regions']]
        average_line_height_list=[]
        average_line_height_not_null=[]
        for m,region in enumerate(regionList):
            baselineCoordsList = [line['baseline'] for line in res['lines'] if line['region']==regionList[m]['pk']]
            midpointList = [[(line[0][0]+line[-1][0])/2,(line[0][1]+line[-1][1])/2] for line in baselineCoordsList]
            lineBegList = [line_intersection([[0,0],[0,pageheight]],[[line[0][0],line[0][1]],[line[-1][0],line[-1][1]]]) for line in baselineCoordsList]
            lineEndList = [line_intersection([[pagewidth,0],[pagewidth,pageheight]],[[line[0][0],line[0][1]],[line[-1][0],line[-1][1]]]) for line in baselineCoordsList]
            d_list=[]
            for n,midpoint in enumerate(midpointList):
                if n>0:
                    p1=array(lineBegList[n-1])
                    p2=array(lineEndList[n-1])
                    p3=array(midpoint)
                    d_list.append(cross(p2-p1,p3-p1)/linalg.norm(p2-p1))
                elif len(midpointList)>1:
                    p1=array(lineBegList[n+1])
                    p2=array(lineEndList[n+1])
                    p3=array(midpoint)
                    d_list.append(-(cross(p2-p1,p3-p1)/linalg.norm(p2-p1)))
            if not(d_list):
                print('one line region')
                average_line_height_list.append(0)
            else:
                average_line_height=int(round(sum(d_list)/len(d_list),0))
                average_line_height_list.append(average_line_height)
                average_line_height_not_null.append(average_line_height)
        print(average_line_height_list)
        print(average_line_height_not_null)
        if not(average_line_height_not_null):
            global_average_line_height=basic_line_height
            print('bad average line height')
        else:
            global_average_line_height=int(round(sum(average_line_height_not_null)/len(average_line_height_not_null),0))
            print(global_average_line_height)

        for m,region in enumerate(regionList):
            if average_line_height_list[m]==0:
                this_line_height=global_average_line_height
            else:
                this_line_height=average_line_height_list[m]
            baselineCoordsList = [line['baseline'] for line in res['lines'] if line['region']==regionList[m]['pk']]
            baselinePKList = [line['pk'] for line in res['lines'] if line['region']==regionList[m]['pk']]
            for n,line in enumerate(baselineCoordsList):
                old_pt=line[0]
                for o,pt in enumerate(line[1:]):
                    #print('region '+str(m)+', line '+str(n)+', point '+str(o)+str(old_pt)+':'+str(pt))
                    angle=get_angle([old_pt,pt])
                    if o==0:
                        inside_above=False
                        inside_below=False
                        new_pt_above=rotateNP((old_pt[0],old_pt[1]-this_line_height*splitfactor),origin=tuple(old_pt),degrees=angle)
                        new_pt_below=rotateNP((old_pt[0],old_pt[1]+this_line_height*(1-splitfactor)),origin=tuple(old_pt),degrees=angle)
                        #print(new_pt_above,pageheight,pagewidth,safetydistance)
                        new_pt_above=check_point_in_image(new_pt_above,pageheight,pagewidth,safetydistance)
                        new_pt_below=check_point_in_image(new_pt_below,pageheight,pagewidth,safetydistance)
                        #print(new_pt_above)
                        polygon_above=[[int(coordinate) for coordinate in new_pt_above]]
                        polygon_below=[[int(coordinate) for coordinate in new_pt_below]]
                    new_pt_above=[int(coordinate) for coordinate in rotateNP((pt[0],pt[1]-this_line_height*splitfactor),origin=tuple(pt),degrees=angle)]
                    new_pt_below=[int(coordinate) for coordinate in rotateNP((pt[0],pt[1]+this_line_height*(1-splitfactor)),origin=tuple(pt),degrees=angle)]
                    new_pt_above=check_point_in_image(new_pt_above,pageheight,pagewidth,safetydistance)
                    new_pt_below=check_point_in_image(new_pt_below,pageheight,pagewidth,safetydistance)
                    if o>0:
                        if inside_above and not(o==len(line[1:])): # if point is inside and is not the last point, # delete previous point, dont insert next point either # and insert instead the intersection of (p-2:p-1) and (p:p+1)
                            replace_pt=list(line_intersection([polygon_above[-3],polygon_above[-2]],[polygon_above[-1],new_pt_above]))
                            del polygon_above[-2:-1]
                            polygon_above.append(replace_pt)
                        if inside_below:
                            replace_pt=list(line_intersection([polygon_below[2],polygon_below[1]],[polygon_below[0],new_pt_below]))
                            del polygon_below[0:1]
                            polygon_below.insert(0,replace_pt)
                        pol=Polygon(polygon_above+polygon_below)
                        inside_above=pol.contains(Point(new_pt_above))
                        inside_below=pol.contains(Point(new_pt_below))
                    polygon_above.append(new_pt_above) # if point is outside add it to boundary
                    polygon_below.insert(0,new_pt_below) # if point is outside add it to boundary
                    old_pt=pt
                if inside_above:
                    del polygon_above[-1] # if point is inside and is last point delete it
                if inside_below:
                    del polygon_below[0] # if point is inside and is last point delete it
                polygon_boundary=polygon_above+polygon_below
                line_url = urljoin(root_url,'api/documents/'+str(doc_nu)+'/parts/'+str(part)+'/lines/'+str(baselinePKList[n])+'/')
                resline = requests.get(line_url,headers=headers).json()
                resline['mask']=polygon_boundary
                data=json.dumps(resline)
                rput = requests.put(line_url,headers=headers,data=data)
                print(m,rput)
    print('done')

def merge2levels_in_third(doc_pk,part_pk,level1,level2,level3):
  lines_without_master_transcription,proportion,empty_lines=get_lines_without_transcription_level(doc_pk,part_pk,level1)
  if proportion>0.3:
    lines_without_level2_transcription,proportion_level2,empty_lines=get_lines_without_transcription_level(doc_pk,part_pk,level2)
    if proportion_verso<0.3:
      print('from level2 to master on part ',part_pk)
      transfer_txt2level(doc_pk,part_pk,level2,level3)
    else:
      print('from level1 to master on part ',part_pk)
      transfer_txt2level(doc_pk,part_pk,level1,level3)
  else:
    print('master exists for part ',part_pk)

def change_all_regions2main(dok_pk):
  part_pk_list,tr_level_list,region_type_list,line_type_list=get_basic_info(doc_pk)
  for n,part_pk in enumerate(part_pk_list):
    region_list=get_region_pk_list(doc_pk,part_pk)
    for region_pk in region_list:
      regions_url = root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+'/blocks/'+str(region_pk)+'/'
      res=requests.get(regions_url,headers=headers).json()
      print(n,res['typology'])
      if not(res['typology']==2):
        res['typology']=2
        data=json.dumps(res)
        res2=requests.put(regions_url,headers=headers,data=data)
        print(regions_url)
        print(res2)

def plot_polygon(Polygon):
  import matplotlib.pyplot as plt

  plt.plot(*Polygon.exterior.xy)
  #plt.plot(*maskSimple.exterior.xy)

def move_line_up_down(baseline,distance,down,pageheight,pagewidth):
  #down=1
  #up=-1
  safety=5
  angle=get_angle(baseline)
  # 0=horizontal
  if abs(angle)<45:
    if baseline[0][0]<baseline[-1][0]:
      vectorx,vectory=get_vector(angle+90*down,distance) #(LTR)
    else:
      vectorx,vectory=get_vector(angle+90*down,distance) #(RTL) (upside down)
  else:
    if baseline[0][1]<baseline[-1][1]:
      vectorx,vectory=get_vector(angle+90*down,distance) #(vertical up)
    else:
      vectorx,vectory=get_vector(angle+90*down,distance) #(vertical down)
  new_baseline=[[max(safety,min(pagewidth-safety,p[0]+vectorx)),max(safety,min(pageheight-safety,p[1]+vectory))] for p in baseline]
  return(new_baseline)

def extend_single_line(baseline,distance,do_left,do_right,pageheight,pagewidth):
  angle=get_angle(baseline)
  # 0=horizontal
  vectorx,vectory=get_vector(angle,distance)
  p1=baseline[0]
  pz=baseline[-1]
  p0=[]
  pend=[]
  if abs(angle)<45:
    if baseline[0][0]<baseline[-1][0]:
      if do_left:
        p0=[[max(1,min(pagewidth,p1[0]-vectorx)),max(1,min(pageheight,p1[1]-vectory))]]
      if do_right:
        pend=[[max(1,min(pagewidth,pz[0]+vectorx)),max(1,min(pageheight,pz[1]+vectory))]]
    else:
      if do_left:
        p0=[[max(1,min(pagewidth,p1[0]+vectorx)),max(1,min(pageheight,p1[1]+vectory))]]
      if do_right:
        pend=[[max(1,min(pagewidth,pz[0]-vectorx)),max(1,min(pageheight,pz[1]-vectory))]]
  else:
    if baseline[0][1]<baseline[-1][1]:
      if do_left:
        p0=[[max(1,min(pagewidth,p1[0]-vectorx)),max(1,min(pageheight,p1[1]-vectory))]]
      if do_right:
        pend=[[max(1,min(pagewidth,pz[0]+vectorx)),max(1,min(pageheight,pz[1]+vectory))]]
    else:
      if do_left:
        p0=[[max(1,min(pagewidth,p1[0]-vectorx)),max(1,min(pageheight,p1[1]-vectory))]]
      if do_right:
        pend=[[max(1,min(pagewidth,pz[0]+vectorx)),max(1,min(pageheight,pz[1]+vectory))]]
  if not(p0==[]):
    baseline=baseline[1:]
  if not(pend==[]):
    baseline=baseline[:-1]
  new_baseline=p0+baseline+pend
  return(new_baseline)


def move_all_lines_of_part_up_down(doc_pk,part_pk,distance=10,down=1,repoly=True, output=False):
  line_list=get_all_full_lines_of_part(doc_pk,part_pk)
  #[[line['pk'],line['order'],line['baseline'],line['typology'],line['region']] for line in data['results']]
  pageheight,pagewidth=page_height_width(doc_pk,part_pk)
  print(part_pk)
  for line in line_list:
    baseline=line['baseline']
    line_id=line['pk']
    new_baseline=move_line_up_down(baseline,distance,down,pageheight,pagewidth)
    if output:
      #print('angle:',int(round(angle,0)))
      print('old: ',baseline)
      print('new: ',new_baseline)
    line_url = root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+ '/lines/' + str(line_id) + '/'
    line_json = requests.get(line_url,headers=headers).json()
    line_json['baseline']=new_baseline
    res_line = requests.put(line_url,headers=headers,data=json.dumps(line_json))
  if repoly:
    url_repoly=root_url+'api/documents/'+str(doc_pk)+'/parts/'+str(part_pk)+'/reset_masks/'
    repoly_res=requests.post(url_repoly,headers=headers)

def calculate_regions_for_unlinked_lines(doc_nu,part,margin_type,paratext_type):
# calculate regions for unlinked lines by taking the minima and maxima of their boundaries
    from shapely.ops import cascaded_union


    parts_url = urljoin(root_url,'api/documents/'+str(doc_nu)+'/parts/'+str(part)+'/')
    print(parts_url)
    res = requests.get(parts_url,headers=headers).json()
    regpoly_list=[]
    line_links_list=[]

    for line_n,lines in enumerate(res['lines']):
        if lines['region']==None:
            line_array=array(lines['mask'])
            try:
                x1 = min(line_array[:,0])
                y1 = min(line_array[:,1])
                x2 = max(line_array[:,0])
                y2 = max(line_array[:,1])
                poly = Polygon([[x1,y1],[x2,y1],[x2,y2],[x1,y2]])
                if len(regpoly_list)==0:
                    regpoly_list.append(poly)
                    line_links_list=[[line_n]]
                else:
                    no_join_found = True
                    for n,poly1 in enumerate(regpoly_list):
                        if poly.intersects(poly1): #poly overlaps with poly1 :
                    # merge: https://deparkes.co.uk/2015/02/28/how-to-merge-polygons-in-python/
                            polygons = [poly1, poly]
                            union_poly = cascaded_union(polygons)
                    # replace poly1 by union_poly
                            regpoly_list[n] = union_poly
                            poly = union_poly
                            line_links_list[n].append(line_n)
                            no_join_found = False
                    if no_join_found:
                        regpoly_list.append(poly)
                        line_links_list.append([line_n])
            except:
                print('error at:',str(part))
    blocks_url=root_url+'api/documents/'+str(doc_nu)+'/parts/'+str(part)+'/'
    data=requests.get(blocks_url,headers=headers).json()
    existing_regions=data['regions']
    post_url = root_url+'api/documents/'+str(doc_nu)+'/parts/'+str(part)+ '/blocks/'
    max_region_n=-1
    if len(existing_regions)==0:
        max_region_size=0
        for region_n,region in enumerate(regpoly_list):
            this_region_area=region.area
            if this_region_area>max_region_size:
                max_region_size=this_region_area
                max_region_n=region_n
        maincol=regpoly_list[max_region_n]
        maincol_point_list = [[int(float(j)) for j in i] for i in list(maincol.exterior.coords)]
        block_json={'document_part': part,'box': maincol_point_list,'typology': 2}
        r = requests.post(post_url,headers=headers, data=json.dumps(block_json))
        y_main_min = min(array(maincol_point_list)[:,1])
        y_main_max = max(array(maincol_point_list)[:,1])
    else:
        y_main_min=10000
        y_main_max=0
        for region in existing_regions:
            if len(region['box'])<3:
                print('delete '+str(region['pk']))
                delete_url=root_url+'api/documents/'+str(doc_nu)+'/parts/'+str(part)+ '/blocks/'+str(region['pk']) + '/'
                r = requests.delete(delete_url,headers=headers)
                print(r.status_code)
            else:
                maincol=Polygon(region['box'])
                y1 = min(array(region['box'])[:,1])
                if y1<y_main_min:
                    y_main_min=y1
                y2 = max(array(region['box'])[:,1])
                if y2>y_main_max:
                    y_main_max=y2
    if max_region_n>-1:
        del regpoly_list[max_region_n] # KICK MAINCOL out of REGPOLYLIST
    for region_n,poly in enumerate(regpoly_list):
        poly_point_list = [[int(float(j)) for j in i] for i in list(poly.exterior.coords)]
        y_poly = mean(array(poly_point_list)[:,1])
        if (y_poly<y_main_min) or (y_poly>y_main_max):
            region_type=paratext_type
        else:
            region_type=margin_type
        block_json={'document_part': part,'box': poly_point_list,'typology': region_type}
        r = requests.post(post_url,headers=headers, data=json.dumps(block_json))
    reorder_url=root_url+'api/documents/'+str(doc_nu)+'/parts/'+str(part)+'/recalculate_ordering/'
    reorder_res = requests.post(reorder_url,headers=headers)


#if __name__ == '__main__':
 # print("<> Running the module as a script! <>")

In [ ]:
#@title import functions
from __init__ import serverconnections
#from basic_api_functions import *
#from complex_api_functions import *
from numpy import rad2deg,deg2rad,array,atleast_2d,cos,sin,arctan2,squeeze,cross,linalg
import numpy
import math
import json
import requests
import os
import re
from requests.compat import urljoin
from shapely.geometry import Point, Polygon
from scipy.optimize import linear_sum_assignment
import collections
from google.colab import files
!pip install bidict
import bidict
!pip install xlsxwriter
import xlsxwriter